In [1]:
from fastai.imports import *
from lemonpie.basics import *
from lemonpie.preprocessing import vocab
from lemonpie.preprocessing.transform import *
from lemonpie.data import *
from lemonpie import models

import torch
import pytorch_lightning as pl
from torchmetrics import MetricCollection, AUROC, Accuracy


In [2]:
COHERENT_DATA_STORE = '/home/vinod/code/datasets/coherent'
COHERENT_DATAGEN_DATE = '08-10-2021'
COHERENT_CONDITIONS = {
    "heart_failure" : "88805009",
    "coronary_heart" : "53741008",
    "myocardial_infarction" : "22298006",
    "stroke" : "230690007",
    "cardiac_arrest" : "410429000"
}

In [3]:
COHERENT_LABELS = list(COHERENT_CONDITIONS.keys())
COHERENT_LABELS

['heart_failure',
 'coronary_heart',
 'myocardial_infarction',
 'stroke',
 'cardiac_arrest']

# Get data

In [4]:
coherent_data = MultimodalEHRData(
    COHERENT_DATA_STORE, 
    COHERENT_LABELS,     
    age_start=240,
    age_range=120,
    start_is_date=False,
    age_in_months=True, 
    lazy_load_gpu=True)

In [5]:
demograph_dims, rec_dims, demograph_dims_wd, rec_dims_wd = vocab.get_all_emb_dims(vocab.EhrVocabList.load(COHERENT_DATA_STORE))
dls, pos_wts = coherent_data.get_data()

In [6]:
coherent_data.modality_types

{'train': ['0', '21', '30', '31', '11', '1', '20', '10'],
 'valid': ['21', '30', '31', '11', '1', '20', '10'],
 'test': ['21', '30', '31', '11', '1', '20', '10']}

In [7]:
dls

{'train': <torch.utils.data.dataloader.DataLoader at 0x7fa4d8c79e20>,
 'valid': <torch.utils.data.dataloader.DataLoader at 0x7fa4d87b6790>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7fa4d8756280>}

In [8]:
train_dl = dls["train"]
valid_dl = dls["valid"]
test_dl = dls["test"]

# Unimodal Models - One Per Modality
These are stubs / dummys to be replaced by the real ones.

In [9]:
class UnimodalModel(pl.LightningModule):
    def __init__(self, input_dims: tuple):
        super().__init__()

        # args
        self.input_dims = input_dims

        # model
        self.model = nn.Sequential(
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 30),
            nn.ReLU(),
            nn.Linear(30, 5),
            nn.Sigmoid()
            )
        
    
    def forward(self, x):
        assert x[0].size() == self.input_dims, f"Expected {self.input_dims}, got {x[0].size()}"
        bs = len(x)
        fake_x = torch.randn((bs, 10), device=self.device)
        return self.model(fake_x)

    def training_step(self, *args, **kwargs):
        return super().training_step(*args, **kwargs)

    def configure_optimizers(self):
        return super().configure_optimizers()



## MRI

### Stub

In [11]:
mri_model = UnimodalModel((4,4))
mri_model(torch.randn(3,4,4)) # batch_size=3

tensor([[0.4827, 0.4370, 0.5291, 0.4324, 0.5247],
        [0.4691, 0.4958, 0.5113, 0.4747, 0.5402],
        [0.4821, 0.4977, 0.5239, 0.4663, 0.5329]], grad_fn=<SigmoidBackward0>)

In [12]:
mri_trainer = pl.Trainer(max_epochs=3, accelerator='gpu', devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [14]:
mri_trainer.fit(mri_model, train_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/core/module.py:1368: UserWarning: `configure_optimizers` must be implemented to be used with the Lightning Trainer
  rank_zero_warn("`configure_optimizers` must be implemented to be used with the Lightning Trainer")
/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/core/optimizer.py:182: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  rank_zero_warn(

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 1.0 K 
-------------------------------------
1.0 K     Trainable params
0         Non-trainable params
1.0 K     Total params
0.004     Total estimated model params size (MB)


Epoch 0:   0%|          | 0/21 [00:00<?, ?it/s] 

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1891: PossibleUserWarning: The number of training batches (21) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:   5%|▍         | 1/21 [00:01<00:26,  1.34s/it, loss=nan, v_num=34]

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/core/module.py:670: UserWarning: `training_step` must be implemented to be used with the Lightning Trainer
  rank_zero_warn("`training_step` must be implemented to be used with the Lightning Trainer")
/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/loops/optimization/optimizer_loop.py:135: UserWarning: `training_step` returned `None`. If this was on purpose, ignore this warning...
  self.warning_cache.warn("`training_step` returned `None`. If this was on purpose, ignore this warning...")


Epoch 2: 100%|██████████| 21/21 [00:03<00:00,  6.81it/s, loss=nan, v_num=34]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 21/21 [00:03<00:00,  6.80it/s, loss=nan, v_num=34]


In [16]:
mri_trainer.save_checkpoint("./mri_model.pth")

### Real MRI Model

In [10]:
# from torchvision.io import read_image
# from torchvision import transforms
from torchvision.models import resnet18
# import ast

```python
class MRIDataset(Dataset):

    def __init__(self, datastore: str) -> None:
        self.data_dir = Path(f"{datastore}/processed/mri")
        self.mri_png_paths = list(self.data_dir.glob('*.png'))  # list of mri slices as png images
        self.img_size = 128

    def find_mri_png_paths(self, patient_id: str) -> list:
        """
        For each patient, find the corresponding paths to the MRI .png images
        and return them in a list
        """
        res = []
        for path in self.mri_png_paths:
            if patient_id in path.name:
                res.append(path)
        return res

    def __getitem__(self, patient_id):
        """
        Return all torch Tensors of MRI slices of a patient and his/her condition labels
        """
        png_paths = self.find_mri_png_paths(patient_id)
        data = []
        for path in png_paths:
            img = read_image(str(path))
            img = img.type(torch.FloatTensor) 
            img = transforms.Resize((self.img_size, self.img_size))(img)
            data.append(img)
        data = torch.stack(data[:115])

        return data

    def __len__(self):
        return len(self.mri_png_paths)
```

In [11]:
mri_ds = MRIDataset(COHERENT_DATA_STORE)

In [25]:
# ptids = ['fec6d99f-1cfd-f397-e740-e3952410ea2a', 'fd83348f-f088-3a44-b8cc-faa955eca3a1', '6dc8bd6b-e2a8-92bf-613d-8b477eb87d7c']
# for ptid in ptids:
#     mri_input = mri_ds[ptid]
#     print(mri_input.shape)

In [26]:
class MRIModel(pl.LightningModule):

    def __init__(self, lr=0.001):
        super().__init__()
        self.lr = lr
        self.resnet = resnet18()
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7),  # change input channel to be 1 instead of 3 
                                      stride=(2, 2), padding=(3, 3), bias=False)
        # add a linear layer at the end for transfer learning
        self.linear = nn.Linear(in_features=self.resnet.fc.out_features,
                                out_features=5)
        self.save_hyperparameters()  # log hyperparameters

    # optionally, define a forward method
    def forward(self, xs):
        pts = xs.shape[0]
        all_y_hats = []
        for pt in range(pts):
            y_hats = self.resnet(xs[pt])
            y_hats = self.linear(y_hats)
            y_hats = torch.mean(y_hats, dim=0)
            all_y_hats.append(y_hats)

        res = torch.stack(all_y_hats)
        return res  # we like to just call the model's forward method
    
    def training_step(self, batch, batch_idx):
        if "mri" in batch.keys():
            xs, ys = batch["mri"], batch["ys"]
            y_hats = self.forward(xs)
            loss = F.binary_cross_entropy_with_logits(y_hats, ys)
            self.log("train_loss", loss, prog_bar=True, on_epoch=True, on_step=True)
            return loss
        else:
            return 

    def validation_step(self, batch, batch_idx):
        if "mri" in batch.keys():
            xs, ys = batch["mri"], batch["ys"]
            y_hats = self.forward(xs)
            loss = F.binary_cross_entropy_with_logits(y_hats, ys)
            self.log("val_loss", loss, prog_bar=True, on_epoch=True, on_step=True)
            return loss
        else:
            return
    
    # def test_step(self, xs, batch_idx):
    #     y_hats = self.resnet(xs)
    #     y_hats = self.linear(y_hats)
    #     return y_hats

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

In [27]:
# mri_model = MRIModel.load_from_checkpoint("./mri_model_ckpt.pth")
# mri_model = torch.load("./mri_model_ckpt.pth")
mri_model = MRIModel()

In [28]:
batch = next(iter(valid_dl))

In [30]:
y_hats = mri_model(batch["mri"])
# mean_y_hats = torch.mean(y_hats, dim=0)
# torch.sigmoid(mean_y_hats)

In [31]:
y_hats

tensor([[ 0.0478, -0.0757, -0.1002,  0.1936, -0.2731],
        [ 0.0477, -0.0718, -0.0998,  0.1903, -0.2689],
        [ 0.0489, -0.0728, -0.0984,  0.1927, -0.2670]],
       grad_fn=<StackBackward0>)

In [32]:
mri_trainer = pl.Trainer(accelerator='gpu', devices=-1, max_epochs=3)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [34]:
# mri_trainer.fit(mri_model, train_dl, valid_dl)

In [23]:
mri_layers_trunc = list(mri_model.children())[:-1]
trunc_mri_model = nn.Sequential(*mri_layers_trunc)

In [35]:
# mri_reprs = trunc_mri_model(mri_input)
# mean_mri_reprs = torch.mean(mri_reprs, dim=0)

In [119]:
mean_mri_reprs.shape

torch.Size([1000])

## DNA

In [17]:
dna_model = UnimodalModel((3,2))
dna_model(torch.randn(2,3,2)) 

tensor([[0.5000, 0.5450, 0.5458, 0.5251, 0.5495],
        [0.4860, 0.5420, 0.5555, 0.5264, 0.5618]], grad_fn=<SigmoidBackward0>)

In [15]:
dna_trainer = pl.Trainer(max_epochs=3, accelerator='gpu', devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [16]:
dna_trainer.fit(dna_model, train_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 1.0 K 
-------------------------------------
1.0 K     Trainable params
0         Non-trainable params
1.0 K     Total params
0.004     Total estimated model params size (MB)


Epoch 2: 100%|██████████| 21/21 [00:02<00:00,  7.26it/s, loss=nan, v_num=31]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 21/21 [00:02<00:00,  7.25it/s, loss=nan, v_num=31]


In [17]:
dna_trainer.save_checkpoint("./dna_model.pth")

## ECG

In [18]:
ecg_model = UnimodalModel((5,))
ecg_model(torch.randn((4,5,)))

tensor([[0.5045, 0.5088, 0.4852, 0.5392, 0.4659],
        [0.5082, 0.5088, 0.4802, 0.5626, 0.4721],
        [0.5293, 0.5082, 0.4893, 0.5951, 0.5158],
        [0.5302, 0.5078, 0.4481, 0.4545, 0.4459]], grad_fn=<SigmoidBackward0>)

In [19]:
ecg_trainer = pl.Trainer(max_epochs=3, accelerator='gpu', devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [20]:
ecg_trainer.fit(ecg_model, train_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 1.0 K 
-------------------------------------
1.0 K     Trainable params
0         Non-trainable params
1.0 K     Total params
0.004     Total estimated model params size (MB)


Epoch 2: 100%|██████████| 21/21 [00:02<00:00,  7.19it/s, loss=nan, v_num=32]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 21/21 [00:02<00:00,  7.18it/s, loss=nan, v_num=32]


In [21]:
ecg_trainer.save_checkpoint("./ecg_model.pth")

## EHR

In [19]:
ehr_model = models.EHR_LSTM(
    demograph_dims,
    rec_dims,
    demograph_dims_wd,
    rec_dims_wd,
    len(COHERENT_LABELS),
    pos_wts["train"], 
    pos_wts["valid"],
    optim="adam",
    base_lr=0.001,
)


/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [22]:
ehr_trainer = pl.Trainer(precision=16, accelerator='gpu', devices=-1, max_epochs=3)
# ehr_output = ehr_trainer.test(ehr_model, test_dl)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [23]:
ehr_trainer.fit(ehr_model, train_dl, valid_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type              | Params
----------------------------------------------------
0 | train_loss_fn | BCEWithLogitsLoss | 0     
1 | valid_loss_fn | BCEWithLogitsLoss | 0     
2 | embs          | ModuleList        | 12.0 K
3 | embgs         | ModuleList        | 19.5 K
4 | input_dp      | InputDropout      | 0     
5 | lstm          | LSTM              | 250 K 
6 | linear        | Sequential        | 7.4 M 
7 | train_metrics | MetricCollection  | 0     
8 | valid_metrics | MetricCollection  | 0     
9 | test_metrics  | MetricCollection  | 0     
----------------------------------------------------
7.7 M     Trainable params
0         Non-trainable params
7.7 M     Total params
15.320    Total estimated model params size (MB)


Epoch 0:   0%|          | 0/28 [00:00<?, ?it/s]                            

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1891: PossibleUserWarning: The number of training batches (21) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 2: 100%|██████████| 28/28 [00:08<00:00,  3.32it/s, loss=1.02, v_num=35]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 28/28 [00:08<00:00,  3.25it/s, loss=1.02, v_num=35]


- [Lightning warning details](https://github.com/Lightning-AI/lightning/issues/10349#issuecomment-961340903)

In [25]:
ehr_trainer.save_checkpoint("./ehr_model.pth")

# Unimodal Datasets

```python
class MRIDataset(torch.utils.data.Dataset):
    def __init__(self, datastore: str, tensor_sz: tuple):
        super().__init__()
        self.mri_dir = f"{datastore}/output/dicom"
        self.tensor_sz = tensor_sz
    
    def __getitem__(self, i):
        mri_fname = glob.glob(f"{self.mri_dir}/*{i}*")
        if len(mri_fname) == 1:
            return torch.full(self.tensor_sz, 1)
        else:
            raise Exception(f"MRI filename match error - found {len(mri_fname)} files with ptid: {i}.")

    def __len__(self):
        return 1
```

```python
class DNADataset(torch.utils.data.Dataset):
    def __init__(self, datastore: str, tensor_sz: tuple):
        super().__init__()
        self.dna_dir = f"{datastore}/output/dna"
        self.tensor_sz = tensor_sz
    
    def __getitem__(self, i):
        dna_fname = glob.glob(f"{self.dna_dir}/*{i}*")
        if len(dna_fname) == 1:
            return torch.full(self.tensor_sz, 10)
        else:
            raise Exception(f"DNA filename match error - found {len(dna_fname)} files with ptid: {i}.")

    def __len__(self):
        return 1
```

```python
class ECGDataset(torch.utils.data.Dataset):
    def __init__(self, datastore: str, tensor_sz: tuple):
        super().__init__()
        ecg_data = pd.read_csv(f"{datastore}/ecg.csv")
        self.ecg_pids = ecg_data.patient.unique()
        self.tensor_sz = tensor_sz
    
    def __getitem__(self, i):
        
        if i in self.ecg_pids:
            return torch.full(self.tensor_sz, 20)
        else:
            raise Exception(f"ptid: {i} - not found in ECG data.")

    def __len__(self):
        return 1
```

# Late Fusion

| Modality Type | Modalities            |   
|---	        |---	                |
| **0**	        | **EHR**               |
| **1**         | EHR + **MRI**         |
| **10**        | EHR + **DNA**         |      
| 11   	        | EHR + MRI + DNA       |
| **20**        | EHR + **ECG**         |
| 21            | EHR + MRI + ECG       |
| 30            | EHR + DNA + ECG       |
| 31            | EHR + MRI + DNA + ECG |


In [25]:
batch = next(iter(valid_dl))   
batch

{'patients': [ptid:1a82483d-7eb2-d5e0-1e1f-398ba129b18b, birthdate:1936-12-22, [('heart_failure', True), ('coronary_heart', False)].., device:cpu,
  ptid:6dc8bd6b-e2a8-92bf-613d-8b477eb87d7c, birthdate:1911-12-23, [('heart_failure', True), ('coronary_heart', False)].., device:cpu,
  ptid:844a37ff-ce26-6338-fd6a-0bc1e925a702, birthdate:1933-03-15, [('heart_failure', True), ('coronary_heart', False)].., device:cpu],
 'ys': tensor([[1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.]]),
 'mri': tensor([[[1, 1, 1, 1],
          [1, 1, 1, 1],
          [1, 1, 1, 1],
          [1, 1, 1, 1]],
 
         [[1, 1, 1, 1],
          [1, 1, 1, 1],
          [1, 1, 1, 1],
          [1, 1, 1, 1]],
 
         [[1, 1, 1, 1],
          [1, 1, 1, 1],
          [1, 1, 1, 1],
          [1, 1, 1, 1]]]),
 'ecg': tensor([[20, 20, 20, 20, 20],
         [20, 20, 20, 20, 20],
         [20, 20, 20, 20, 20]])}

In [37]:
torch.sigmoid(ehr_model(batch["patients"]))

tensor([[0.5964, 0.5248, 0.4754, 0.4943, 0.6138],
        [0.5518, 0.5288, 0.5270, 0.5399, 0.5448],
        [0.5626, 0.4570, 0.4899, 0.4378, 0.6217]], grad_fn=<SigmoidBackward0>)

In [30]:
mri_input, ecg_input = batch["mri"], batch["ecg"]

In [31]:
mri_model(mri_input)

tensor([[0.4853, 0.4834, 0.5167, 0.4784, 0.5325],
        [0.5083, 0.5090, 0.5161, 0.4914, 0.5421],
        [0.5048, 0.4745, 0.5100, 0.4634, 0.5208]], grad_fn=<SigmoidBackward0>)

In [32]:
ecg_model(ecg_input)

tensor([[0.5326, 0.5161, 0.4775, 0.5335, 0.4699],
        [0.5141, 0.5114, 0.4913, 0.5569, 0.4736],
        [0.5312, 0.4955, 0.4771, 0.5711, 0.4843]], grad_fn=<SigmoidBackward0>)

In [39]:
def run_late_fusion(dl, verbose=False):

    all_batches = []
    total_length = 0

    for batch in dl:

        batch_output = []
        pts = batch["patients"]
        
        total_length += len(pts)
        if verbose:
            print(f"batch size: {len(pts)}, first ptid: {pts[0].ptid}")

        batch_output.append(torch.sigmoid(ehr_model(pts)))

        # other modalities 
        if "mri" in batch.keys():
            batch_output.append(mri_model(batch["mri"]))
        if "dna" in batch.keys():
            batch_output.append(dna_model(batch["dna"]))
        if "ecg" in batch.keys():
            batch_output.append(ecg_model(batch["ecg"]))

        # avg across multimodal models for every patient in batch
        avgd = torch.mean(torch.stack(batch_output), dim=0)         
        all_batches.append(avgd)


    print(f"Completed {total_length} patients.")

    # flatten across batches - list of lists (batches) to flat
    output = [pt for batch in all_batches for pt in batch]
    return output

In [40]:
test_result = run_late_fusion(valid_dl)

Completed 128 patients.


In [45]:
valid_result = run_late_fusion(valid_dl, verbose=True)

batch size: 3, first ptid: 1a82483d-7eb2-d5e0-1e1f-398ba129b18b
batch size: 13, first ptid: 2e1cf98c-70ce-4f8f-36da-b2eef4960ecc
batch size: 2, first ptid: 4a1a224f-54d6-66f1-3755-4c8489f2a5de
batch size: 14, first ptid: aead835e-66f2-d3b9-099c-c33844a70748
batch size: 12, first ptid: 972f6a59-3921-36bc-64bb-253d6241748b
batch size: 34, first ptid: e1875ec6-e10f-c9d8-d388-fd3abfbc4a87
batch size: 50, first ptid: 1e73e6da-68f0-0ffc-4062-a4647b5b67c4
Completed 128 patients.


In [46]:
train_result = run_late_fusion(train_dl)

Completed 1022 patients.


In [47]:
assert len(test_result) == len(valid_result) == 128
assert len(train_result) == 1022

# Joint Fusion

In [48]:
pts, y, m, other = next(iter(valid_dl))

In [49]:
pts

'patients'

In [50]:
m[0]

'm'

## MRI

In [52]:
mri_model = UnimodalModel.load_from_checkpoint("./mri_model_pth", input_dims=(4,4))

In [53]:
mri_layers_trunc = list(mri_model.model.children())[:-2]

In [54]:
mri_model.model = nn.Sequential(*mri_layers_trunc)

In [55]:
# mri_model

In [56]:
mri_repr = mri_model(mri_input)

In [57]:
mri_repr.shape

torch.Size([3, 30])

## EHR

In [58]:
ehr_model = models.EHR_LSTM.load_from_checkpoint(
    "./ehr_model_pth",
    demograph_dims=demograph_dims,
    rec_dims=rec_dims,
    demograph_dims_wd=demograph_dims_wd,
    rec_dims_wd=rec_dims_wd,
    labels=len(COHERENT_LABELS),
    train_pos_wts=pos_wts["train"], 
    valid_pos_wts=pos_wts["valid"],
    optim="adam",
    base_lr=0.001,
)

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [59]:
ehr_layers_trunc = list(ehr_model.linear.children())[:-2]

In [60]:
ehr_model.linear = nn.Sequential(*ehr_layers_trunc)

In [61]:
# ehr_model

In [62]:
ehr_repr = ehr_model(pts)

In [63]:
ehr_repr.shape

torch.Size([3, 3328])

## Joint Model

In [64]:
concated = torch.concat((ehr_repr, mri_repr), dim=1)

In [65]:
concated.shape

torch.Size([3, 3358])

In [51]:
class JointFusion(pl.LightningModule):
    def __init__(self,
            demograph_dims,
            rec_dims,
            demograph_wd,
            rec_wd,
            num_labels,
            train_pos_wts,
            valid_pos_wts
            ):

        super().__init__()

        ## args
        self.train_loss_fn = nn.BCEWithLogitsLoss(pos_weight=train_pos_wts)
        self.valid_loss_fn = nn.BCEWithLogitsLoss(pos_weight=valid_pos_wts)


        # ehr
        self.ehr_model = models.EHR_LSTM.load_from_checkpoint(
            "./ehr_model.pth",
            demograph_dims=demograph_dims,
            rec_dims=rec_dims,
            demograph_dims_wd=demograph_dims_wd,
            rec_dims_wd=rec_dims_wd,
            labels=len(COHERENT_LABELS),
            train_pos_wts=pos_wts["train"], 
            valid_pos_wts=pos_wts["valid"],
            optim="adam",
            base_lr=0.001,
        )
        ehr_layers_trunc = list(self.ehr_model.linear.children())[:-2]
        self.ehr_model.linear = nn.Sequential(*ehr_layers_trunc)

        # mri
        self.mri_model = UnimodalModel.load_from_checkpoint("./mri_model.pth", input_dims=(4,4))
        mri_layers_trunc = list(self.mri_model.model.children())[:-2]
        self.mri_model.model = nn.Sequential(*mri_layers_trunc)
        
        # dna
        self.dna_model = UnimodalModel.load_from_checkpoint("./dna_model.pth", input_dims=(3,2))
        dna_layers_trunc = list(self.dna_model.model.children())[:-2]
        self.dna_model.model = nn.Sequential(*dna_layers_trunc)

        # ecg
        self.ecg_model = UnimodalModel.load_from_checkpoint("./ecg_model.pth", input_dims=(5,))
        ecg_layers_trunc = list(self.ecg_model.model.children())[:-2]
        self.ecg_model.model = nn.Sequential(*ecg_layers_trunc)

        # model
        self.repr_dims = {}
        self.repr_dims["ehr"] = self.ehr_model.repr_dim
        self.repr_dims["mri"] = 30
        self.repr_dims["dna"] = 30
        self.repr_dims["ecg"] = 30
        self.repr_dims["total"] = sum(self.repr_dims.values())

        self.model = nn.Sequential(nn.Linear(self.repr_dims["total"], 4000),
                      nn.ReLU(),
                      nn.Linear(4000, 5000),
                      nn.ReLU(),
                      nn.Linear(5000, 5),
                      nn.Sigmoid())
        
        ## metrics
        metrics = MetricCollection(
            [
                # Accuracy(),
                AUROC(num_classes=num_labels, pos_label=1, average="micro"),
                # Recall(),
                # Precision(),
                # AveragePrecision(num_classes)
            ]
        )
        self.train_metrics = metrics.clone(prefix="train/")
        self.valid_metrics = metrics.clone(prefix="valid/")
        self.test_metrics = metrics.clone(prefix="test/")


    
    def forward(self, batch):
        pts = batch["patients"]
        pts = [pt.to_gpu(non_block=True) for pt in pts]

        concated_reprs = torch.zeros((len(pts), self.repr_dims["total"]), device=self.device)

        # ehr
        concated_reprs[:, :self.ehr_model.repr_dim] = self.ehr_model(pts)
            
        # other modalities 
        if "mri" in batch.keys():
            # mri_input.to(self.device)
            concated_reprs[:, 3328:3358] = self.mri_model(batch["mri"])
        if "dna" in batch.keys():
            # dna_input.to(self.device)
            concated_reprs[:, 3358:3388] = self.dna_model(batch["dna"])
        if "ecg" in batch.keys():
            # ecg_input.to(self.device)
            concated_reprs[:, 3388:] = self.ecg_model(batch["ecg"])

        # send through fusion
        # concated_reprs.to(self.device, non_blocking=True)
        return self.model(concated_reprs)

    def training_step(self, batch, batch_idx):
        yb = batch["ys"]
        y_hat = self(batch)
        train_loss = self.train_loss_fn(y_hat, yb)

        self.log("train_loss", train_loss, on_step=True, on_epoch=True)
        self.train_metrics.update(y_hat, yb.int())
        self.log_dict(self.train_metrics.compute(), on_step=False, on_epoch=True)

        return train_loss

    def validation_step(self, batch, batch_idx):
        yb = batch["ys"]
        y_hat = self(batch)
        valid_loss = self.valid_loss_fn(y_hat, yb)

        self.log("valid_loss", valid_loss, on_step=True, on_epoch=True)
        self.valid_metrics.update(y_hat, yb.int())
        self.log_dict(self.valid_metrics.compute(), on_step=False, on_epoch=True)

        return valid_loss

    def test_step(self, batch, batch_idx):
        yb = batch["ys"]
        y_hat = self(batch)
        
        self.test_metrics.update(y_hat, yb.int())
        self.log_dict(self.test_metrics.compute(), on_step=False, on_epoch=True)

        return 
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters())
        return optimizer



In [52]:
fusion_model = JointFusion(
    demograph_dims,
    rec_dims,
    demograph_dims_wd,
    rec_dims_wd,
    len(COHERENT_LABELS),
    pos_wts["train"], 
    pos_wts["valid"])

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [53]:
fusion_trainer = pl.Trainer(precision=16, accelerator='gpu', devices=-1, max_epochs=5)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [54]:
fusion_trainer.fit(fusion_model, train_dataloaders=valid_dl)

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:117: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type              | Params
----------------------------------------------------
0 | train_loss_fn | BCEWithLogitsLoss | 0     
1 | valid_loss_fn | BCEWithLogitsLoss | 0     
2 | ehr_model     | EHR_LSTM          | 7.6 M 
3 | mri_model     | UnimodalModel     | 850   
4 | dna_model     | UnimodalModel     | 850   
5 | ecg_model     | UnimodalModel     | 850   
6 | model         | Sequential        | 33.7 M
7 | train_metrics | MetricCollection  | 0     
8 | valid_metrics | MetricCollection  | 0     
9 | test_metrics  | MetricCollection  | 0     
----------------------------------------------------
41.4 M    Trainable params
0         Non-trainable params
41.4 M    Total params
82.704    T

Epoch 0:   0%|          | 0/7 [00:00<?, ?it/s] 

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1891: PossibleUserWarning: The number of training batches (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 4: 100%|██████████| 7/7 [00:02<00:00,  3.18it/s, loss=1.01, v_num=36] 

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 7/7 [00:03<00:00,  2.24it/s, loss=1.01, v_num=36]


In [55]:
fusion_trainer.test(fusion_model, test_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 8/8 [00:00<00:00, 16.81it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test/AUROC         │    0.5192548036575317     │
└───────────────────────────┴───────────────────────────┘

[{'test/AUROC': 0.5192548036575317}]